<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/basic_examples/tour_of_lsdtt_basic_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A tour of lsdtt-basic-metrics


Last updated by Simon M Mudd on 15/12/2023

`lsdtt-basic-metrics` is one of the command line tools included in *LSDTopoTools*. It has the greatest number of analysis options and scrolling through it all can be a bit daunting. This is a tour through some of its functionality. 

If you use this software for your research, please cite https://doi.org/10.5281/zenodo.3245040

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **First get data** section. 

First we install `lsdtopotools`. The first line downloads the package and the second installs it. The `/dev/null` stuff is just to stop the notebook printing a bunch of text to screen.  

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.9.

In [ ]:
!lsdtt-basic-metrics -v

Now we install `lsdviztools`:

In [ ]:
!pip install lsdviztools  &> /dev/null

## Download some data

We need to get some data to download. We are going to download data using the opentopography scraper that is included with `lsdviztools`. You will need to get an opentopography.org account and copy in your API key.

You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo 

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "Diablo"
source_name = "COP30"

SB_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [35.1920215020742,-120.90387764783046], 
                        upper_right_coordinates = [35.296562615076155, -120.73735491824398],
                        prefix = Dataset_prefix, 
                        api_key_file = your_OT_api_key_file)
SB_DEM.print_parameters()
SB_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

## A simple hillshade image

We can run `lsdtt-basic-metrics` with a little interface in `lsdviztools` called the `lsdtt_driver`. Here we are only going to write a hillshade raster.  

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

We can plot the image using a `lsdviztools` function. 

In [ ]:
%matplotlib inline
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology",dpi=500)

You can save the figure as well. This allows you to show it in this notebook in a larger format:

In [ ]:
%%capture 
# Use the capture comment to get rid of all the text
# But it will also not display an inline image
# So you need to call the image from the next line of code           
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", 
                                 save_fig=True, size_format="geomorphology",dpi=600)

In [ ]:
print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

Note that this `.png` file is now stored in your google drive and you can download these images if you want to use them in talks or papers or for any other purpose. 

In [ ]:
!ls *.png

## Now some simple surface metrics

One common topographic analysis is to extract gradients and curvatures from topographic data. Here is how to do this using `lsdtt-basic-metrics`:

In [ ]:
# IMPORTANT all the parameter values must be passed as strings. 
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"write_hillshade" : "true",
                    "surface_fitting_radius" : "60",
                    "print_slope" : "true",
                    "print_curvature" : "true",
                    "print_profile_curvature" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

We can check to see what raster we got by listing everything with the `.bil` extension. We can see a 

* curvature raster (filename has `CURV`)
* slope raster (filename has `SLOPE`)
* profile curvature raster (filename has `PROFCURV`)

*LSDTopoTools* uses a polynomial fitting window to calculate these metrics, the size of which you can set with `surface_fitting_radius`

Other options are

`print_smoothed_elevation`, `print_aspect`, `print_planform_curvature`, `print_tangential_curvature`, `print_tangential_curvature`, `print_directional_gradients`.

In [ ]:
!ls *.bil

### Plot the surface metrics

We will use `lsdviztools` to plot some of the surface metrics. First the slope (or gradient), which `lsdtopotools` identifies with `_SLOPE` in the filename. 

In [ ]:
%%capture             
Base_file = r_prefix

### Plot the topographic gradient
Drape_prefix = r_prefix+"_SLOPE"
this_slope_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="inferno", 
                                       cbar_label = "Gradient (m/m)",
                                       save_fig=True, size_format="geomorphology",dpi=600)

In [ ]:
print(this_slope_img)
from IPython.display import display, Image
display(Image(filename=this_slope_img, width=800))

Now the curcature, which `lsdtopotools` identifies with `_CURV` in the filename. 

In [ ]:
%%capture             
Base_file = r_prefix

### Plot the curvature
Drape_prefix = r_prefix+"_CURV"
this_curv_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="RdBu", 
                                       cbar_label = "Curvature (1/m)",
                                       cbar_loc = "top",
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_curv_img)
from IPython.display import display, Image
display(Image(filename=this_curv_img, width=800))

## Getting drainage area

There are several options for getting drainage area. Here we will show the D-infinity algorithm introducted by Tarboton 1991. The raster has the string `dinf_area` in the filename. 
Other options are:
`print_d8_drainage_area_raster`, `print_QuinnMD_drainage_area_raster`, `print_FreemanMD_drainage_area_raster`, `print_MD_drainage_area_raster`

The rasters from these other kinds of drainage area extractions have filenames that should be obvious (e.g., `d8_area`)


In [ ]:
lsdtt_parameters = {"print_dinf_drainage_area_raster" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!ls *.bil

So, to see drainage area you would really want a logarithmic colour bar. I will first show you how to plot a raster using `rasterio` and `imshow`.

In [ ]:
%matplotlib inline
import rasterio
from matplotlib import pyplot
from matplotlib.colors import LogNorm

fig, ax = pyplot.subplots()
src = rasterio.open('Diablo_COP30_UTM_dinf_area.bil')
cs = ax.imshow(src.read(1), cmap='pink', norm=LogNorm(vmin=100, vmax=100000000))

cbar = fig.colorbar(cs)

cbar.ax.minorticks_off()

The issue with this method is that the axes are in units of pixels rather than the spatial dimension. It is possible to plot this using the actual coordinates, see the `draiange_extraction_with_lsdtopotools` notebook. 

However, `lsdviztools` is optimised to make nice figures, so lets just use that. 

In [ ]:
!ls Diablo*

In [ ]:
### Plot the drainage area
Dataset_prefix = "Diablo"
source_name = "COP30"

Base_file = r_prefix
Drape_prefix = r_prefix+"_dinf_area"
this_area_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="pink", 
                                       cbar_label = "Drainage area",
                                       cbar_loc = "bottom",
                                       drape_cnorm = "LogNorm",
                                       colour_min_max = [1000,10000000],
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_area_img)
from IPython.display import display, Image
display(Image(filename=this_area_img, width=800))